Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC3670 -	Procesamiento de Lenguaje Natural<br>
Segundo Semestre 2025<br>


<h1><center>Tarea 3: Generación de Texto </center></h1>
        Profesor: Marcelo Mendoza<br>
        Fecha de entrega: 17 de octubre


---

## **Integrantes**:
* Integrante1: Ignacio Vergara Briones
  
* Integrante2: Diego Larraguibel Ipinza

* Diego Fernandez Perez de Tudela

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas.
- El notebook debe tener todas las celdas de código ejecutadas.
- Cualquier instancia de copia resultará en un 1,1 como nota de curso.
- Esta tarea requiere uso de GPU. Ir a "Entorno de ejecución" --> "Cambiar tipo de entorno de ejecución" --> Marcar "GPU T4".

---

# Introducción



En cuanto a la estructura de esta segunda tarea, se consideran 4 secciones diferentes.
* Parte 1: Generación de texto con LLaMA 3.1
* Parte 2: Fine-Tune con modelo T5
* Parte 3: Preguntas teóricas


# Librerías

### PARTE 1

In [23]:
import os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from huggingface_hub import login
import evaluate

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
assert torch.backends.mps.is_available()
device = torch.device("mps")

### PARTE 2

# Parte 1: Generación de Texto con modelo Base (15 puntos)

### 1.1 Investigue el modelo https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct. Hint: Qué tarea hace, cuantos parámetros tiene, etc.

**RESPUESTA:**

Es un modelo multilingual de 8 billones de parámetros (creado por Meta) y su objetivo es generar textos en múltiples idiomas. También existen otros modelos con más parámetros (70B y 405B). Este modelo, denominado Instruct, ha sido entrenado con instrucciones para desempeñarse bien en tareas conversacionales, de razonamiento, redacción y asistencia general.

### 1.2 Cargue el modelo previamente mencionado (entregue el token al modelo)

**RESPUESTA:**

In [3]:
login("secret_key")

In [4]:
tok = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    torch_dtype=torch.float16, # Bajé a float16 para que quepa en memoria
    low_cpu_mem_usage=True
).to(device)

if model.generation_config.pad_token_id is None and tok.pad_token_id is None:
    model.generation_config.pad_token_id = tok.eos_token_id


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [9]:
messages = [{"role": "user", "content": "Did I load the LLa;ama 3 model correctly?"}]
inputs = tok.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True
)

inputs = {k: v.to(model.device) for k, v in inputs.items()}

gen = model.generate(
    **inputs,
    max_new_tokens=40,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

prompt_len = inputs["input_ids"].shape[-1]
print(tok.decode(gen[0, prompt_len:], skip_special_tokens=True))

To help you determine if you've loaded the LLaMA 3 model correctly, I'll need more information about your setup and environment. However, I can guide you through some general steps to check


### 1.3 Haga un prompt para resumir texto y otro para traducir texto de español a inglés

**RESPUESTA:**

In [17]:
summary_prompt = "Summarize the following paragrapg in a single concise sentence:"
translation_prompt = "Translate the following paragraph to English:"

### 1.4 Investigue sobre las métricas ROUGE y BLEU. ¿Qué miden? ¿En qué casos es mejor usar una u otra? Compare ambas métricas. Además investigué otra métrica que podría ser útil para evaluar este tipo de modelos.

**RESPUESTA:**

Las métricas ROUGE y BLEU son medidas utilizadas principalmente para evaluar la calidad de textos generados por máquinas, y tienen roles complementarios.

* BLEU mide la precisión del texto generado: cuántas palabras o secuencias de palabras (n-gramas) del texto producido aparecen también en el texto de referencia. Es decir, revisa coincidencias exactas de secuencias de palabras tokenizadas.

* Rouge mide cuántas de las palabras o n-gramas relevantes del texto de referencia aparecen en el texto generado. En algún sentido, es la tarea complementaria a BLEU. Un valor alto de ROUGE indica que el texto generado recupera muchas de las ideas o frases importantes del texto original.

BLEU es una métrica muy útil para tareas como traducción automática, donde las coincidencias exactas del texto producido indica que se están preservando adecuadamente el significado y la estructura del texto original, mostrando que el modelo logra generar traducciones precisas y coherentes en relación con la referencia humana. En contraste, Rouge hace mucho más sentido para tareas como generar resúmenes, donde se espera que chunks del texto original aparezcan en el resumen.

### 1.5 Utilice los 2 prompts sobre el siguiente texto

Isaac Asimov, un escritor de ciencia ficción, previó los peligros potenciales de los agentes autónomos de IA en 1942, mucho antes del desarrollo de los sistemas de IA. Creó las Tres Leyes de la Robótica como un medio para limitar esos riesgos. En el código de ética de Asimov, la primera ley prohíbe a los robots dañar activamente a los humanos o permitir que se les haga daño negándose a actuar. La segunda ley ordena a los robots obedecer a los humanos a menos que las órdenes no estén de acuerdo con la primera ley. La tercera ley ordena a los robots protegerse a sí mismos en la medida en que hacerlo esté de acuerdo con las dos primeras leyes.

***Bibliografía:*** https://www.computerweekly.com/es/definicion/Que-es-la-etica-para-la-IA

**RESPUESTA:**

In [24]:
text = """
Isaac Asimov, un escritor de ciencia ficción, previó los peligros potenciales de los agentes autónomos de IA en 1942, mucho antes del desarrollo de los sistemas de IA. Creó las Tres Leyes de la Robótica como un medio para limitar esos riesgos. En el código de ética de Asimov, la primera ley prohíbe a los robots dañar activamente a los humanos o permitir que se les haga daño negándose a actuar. La segunda ley ordena a los robots obedecer a los humanos a menos que las órdenes no estén de acuerdo con la primera ley. La tercera ley ordena a los robots protegerse a sí mismos en la medida en que hacerlo esté de acuerdo con las dos primeras leyes.
"""

In [25]:
summ_message = [{"role": "user", "content": f"{summary_prompt}\n\n{text}"}]

inputs = tok.apply_chat_template(
    summ_message,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True
)

inputs = {k: v.to(model.device) for k, v in inputs.items()}

gen = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

prompt_len = inputs["input_ids"].shape[-1]
summary_out = tok.decode(gen[0, prompt_len:], skip_special_tokens=True)
print(summary_out)

Isaac Asimov creó las Tres Leyes de la Robótica en 1942 para limitar los riesgos potenciales de los agentes autónomos de IA.


In [26]:
translation_message = [{"role": "user", "content": f"{translation_prompt}\n\n{text}"}]

inputs = tok.apply_chat_template(
    translation_message,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True
)

inputs = {k: v.to(model.device) for k, v in inputs.items()}

gen = model.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

prompt_len = inputs["input_ids"].shape[-1]
translation_out = tok.decode(gen[0, prompt_len:], skip_special_tokens=True)
print(translation_out)

La traducción al inglés del párrafo es:

Isaac Asimov, a science fiction writer, foresaw the potential dangers of autonomous AI agents in 1942, long before the development of AI systems. He created the Three Laws of Robotics as a means to limit those risks. In Asimov's code of ethics, the first law prohibits robots from actively harming humans or allowing them to be harmed by refusing to act. The second law orders robots to obey humans unless the orders conflict with the first law. The third law orders robots to protect themselves as long as doing so is in accordance with the first two laws.


### 1.6 Aplicar las métricas sobre los 2 outputs según corresponda. Comente los resultados obtenidos.

Referecia RESUMEN (realizado por un modelo GPT): Isaac Asimov anticipó los riesgos de la IA en 1942 y formuló las Tres Leyes de la Robótica: los robots no deben dañar a los humanos, deben obedecer sus órdenes salvo que contradigan la primera ley, y deben protegerse a sí mismos siempre que no entren en conflicto con las dos primeras.

Referencia TRADUCCIÓN (realizado por traductor de Google): Isaac Asimov, a science fiction writer, foresaw the potential dangers of autonomous AI agents in 1942, long before the development of AI systems. He created the Three Laws of Robotics as a means of limiting those risks. In Asimov's code of ethics, the first law prohibits robots from actively harming humans or allowing harm to come to humans by refusing to act. The second law commands robots to obey humans unless the orders are inconsistent with the first law. The third law commands robots to protect themselves as long as doing so is consistent with the first two laws.

In [30]:
summary_ref = "Isaac Asimov anticipó los riesgos de la IA en 1942 y formuló las Tres Leyes de la Robótica: los robots no deben dañar a los humanos, deben obedecer sus órdenes salvo que contradigan la primera ley, y deben protegerse a sí mismos siempre que no entren en conflicto con las dos primeras."
translation_ref = "Isaac Asimov, a science fiction writer, foresaw the potential dangers of autonomous AI agents in 1942, long before the development of AI systems. He created the Three Laws of Robotics as a means of limiting those risks. In Asimov's code of ethics, the first law prohibits robots from actively harming humans or allowing harm to come to humans by refusing to act. The second law commands robots to obey humans unless the orders are inconsistent with the first law. The third law commands robots to protect themselves as long as doing so is consistent with the first two laws."

**RESPUESTA:**

In [31]:
# BLEU
bleu = evaluate.load("bleu")
results_bleu = bleu.compute(predictions=[translation_out], references=[translation_ref])
print("BLEU:", results_bleu)

# ROUGE
rouge = evaluate.load("rouge")
results_rouge = rouge.compute(predictions=[summary_out], references=[summary_ref])
print("ROUGE:", results_rouge)

BLEU: {'bleu': 0.7540462944452592, 'precisions': [0.8521739130434782, 0.7807017543859649, 0.7256637168141593, 0.6696428571428571], 'brevity_penalty': 1.0, 'length_ratio': 1.0648148148148149, 'translation_length': 115, 'reference_length': 108}


ROUGE: {'rouge1': 0.41025641025641024, 'rouge2': 0.23684210526315788, 'rougeL': 0.28205128205128205, 'rougeLsum': 0.28205128205128205}


### 1.7 Comente los resultados anteriores

**RESPUESTA:**

Se obtuvo un valor de ```BLEU``` de 0.75, el cual es bastante bueno, pues la escala va de 0 a 1. Esto indica que la traducción del modelo se parece bastante a la referencia humana. El diccionario de precissions corresponde a las precisiones de 1-gram a 4-gram, es decir, qué fracción de n-gramas del output aparecen en la referencia. Que el 4-gram esté en 0.67 sugiere buen orden y frases similares, no solo palabras sueltas.


Respecto a ```ROUGE```, los resultados muestran un nivel moderado de coincidencia. El valor de ROUGE-1 **(0.41)** indica que aproximadamente el 41 % de las palabras del resumen humano aparecen también en el generado, lo que refleja una buena cobertura del contenido principal. El ROUGE-2 (0.24) por su parte, sugiere que $\frac{1}{4}$ de las combinaciones de dos palabras consecutivas coinciden, lo que indica que hay cierta concordancia entre la referencia y el output. Finalmente, el ROUGE-L (0.28) indica un 28 % de solapamiento en las secuencias más largas compartidas entre ambos textos, lo que enfatiza más aún que hay cierta concordancia entre ambos textos.

# Parte 2: Modelo T5 (30 puntos)

### 2.1 Cargue tres modelos T5 (librería transformers) usando:
* Pretrained Model/ t5-base
* Encoder Model (T5 Encoder)
* Conditional Generation Model (T5 For Conditional Generation)

Para cada uno use un ejemplo.

**RESPUESTA:**

### 2.2 ¿Cuáles son las diferencias entre los tres modelos?

**RESPUESTA:**

### 2.3 Utilice el dataset para entrenar sobre la tarea de traducción. El dataset a utilizar se encuentra en canvas. Debe evaluar con una de las métricas anteriores vistas en la sección 1.4.

El dataset original se encuentra en el link: https://www.kaggle.com/datasets/dhruvildave/en-fr-translation-dataset. En canvas se encuentra un dataset reducido para poder hacer el entrenamiento. (HINT: recuerde separar la data en train y test y estudie los datos que les entrega el dataset)

**RESPUESTA:**

### 2.4 Realice una visualización sobre el train_loss con respecto a las epochs

**RESPUESTA:**

### 2.5 Comente sobre los resultados obtenidos en entrenamiento y con respecto a la métrica que usó para evaluación

**RESPUESTA**

### 2.6 Cargue el modelo OPUS https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-fr. Seleccionen una muestra del dataset (testing) de minimo tamaño 3 y realicen inferencias sobre este modelo, el modelo LLaMA y el modelo entrenado previamente.

### 2.7 Haga una comparación entre los resultados de los tres modelos.

**RESPUESTA:**

# Parte 3: Preguntas teóricas (15 puntos)


Para esta sección deberá ingresar a la siguiente página https://poloclub.github.io/transformer-explainer/ y responder las siguientes preguntas. **Es importante utilizar referencias**, basta con dejar el link en la pregunta.

1. ¿Qué ocurre en la primera etapa del proceso del bloque del Transformer? (La parte inicial está a la izquierda)

**RESPUESTA:**

2. ¿Qué son y qué realizan Q, K y V en el transformer?

**RESPUESTA:**

3. ¿Para qué sirve el Positional Encoding? ¿Qué tipos hay? Explique el funcionamiento de al menos 2 tipos

**RESPUESTA:**

4. Explique el funcionamiento del multi-head desde la entrada de los datos hasta el output. (Puede y se recomienda usar de guía la página para la explicación)

**RESPUESTA:**

5. Explique sobre las conexiones residuales, normalización de las capas y el dropout.

**RESPUESTA:**

6. ¿Qué es temperatura, top_k y top_p en los transformers? Explique

**RESPUESTA:**

7. Coloque la siguiente frase: 'The NLP course explains' y genere las siguientes palabras con los siguientes **PARÁMETROS**: **TEMPERATURA** = 1 y **top_k** = 10. Genere un mínimo de 3 palabras y escribalos. Luego cambie la **TEMPERATURA** = 0 y **top_k** = 0 y genere un mínimo de 3 palabras. Finalmente coloque **TEMPERATURA** = 2 y **top_k** = 30. Escriba las palabras generadas para cada caso junto a su probabilidad y luego comente los resultados.

**RESPUESTA:**